# Projekt 6 - Mindenre válaszoló gép

A vidámparkok gyakori automatája szokott lenni egy olyan gép, amibe ha pénzt dobunk és felteszünk neki egy kérdést (legtöbbször a jövőnkkel kapcsolatban), akkor ad rá valami választ. 

Arra gondoltatok, hogy megviccelitek a barátaitokat és készítetek egy ilyen gépet ami képes választ adni bármilyen kérdésre. Persze, azt nem közlitek velük, hogy ennek a válasznak nem lesz semmilyen tudományosan megalapozott háttere vagy érzelmi töltete. Lényegében teljesen véletlenszerű válaszokat fog generálni, ami kérdéstől függően akár elég vicces szituációba hozhatja a barátotokat. 

## Mit fogsz készíteni?

Egy SG90 típusú szervo motorból és 1 nyomógombból álló rendszert építünk, amit a Raspberry Pi-hoz kapcsolunk. A nyomógombbal állítjuk el a programunkat, ha be akarjuk fejezni a játékot. A motor tengelyére egy kart szerelünk, ami a mögötte álló papírlapra rajzolt válaszok közül fog egyet kiválasztani, miután begépeltünk egy kérdést. Három válasz lehetőség lesz a papírlapon, igen, nem, talán. Tehát, olyan kérdéseket vár a gép, amikre ilyen válaszokat lehet adni. Ha nem ilyen jellegű kérdést kapna, akkor kiírja a képernyőre, hogy a kérdés jellege nem jó. **Megjegyzés: Mindig használjunk megfelelő drivert a motorral összekötve, hogy csökkentsük bármelyik eszköz károsodásának lehetőségét.**

## Mit tanulsz meg?

A válaszológép elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy szervo motort egy áramkörbe beiktatni egy nyomó gombbal együtt.
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal.
* **continue** használatát.
* Hogyan készítsünk fel egy lista tartalmát az elemzésre, pl. hogy tartalmaz-e egy előre definiált elemet.
* String műveleteket: *lower(), unidecode(), replace(), split()*.
* Egy listából egy random elemet kiválasztani.
* Szöveg beolvasása változóba *input*-tal.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Inicializálni a Servo, AngularServo és nyomógomb objektumokat.
* Letesztelni a szervo mozgástartományát szögekben, és avval inicializálni az AngularServo-t.
* Definiálni az igen, nem és talán válaszokhoz tartozó szögeket és egy listában tárolni.
* Definiálni egy listában azokat a kérdőszavakat, amik nem felelnek nekünk meg, hogy kiszűrjük őket.
* Definiálni egy függvényt ami kilép a pythonból.
* Definiálni egy függvényt, ami kiszűri, hogy a begépelt kérdés megfelelő típusú-e.
* Definiálni egy függvényt ami a három válasz közül random választ egyet.
* Nyomógombhoz hozzárendelni a kilépés függvényt.
* Végtelen ciklusban beolvasni a kérdést, leellenőrízni, hogy megfelel-e, ha igen válaszolni, ha nem, akkor kiírni, hogy rossz típusú kérdés.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) SG90 Servo motor: [itt vásárolhatsz](https://www.tme.eu/hu/details/oky8003/szervomotorok/okystar/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) 1 db nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

e) 4 db AAA vagy AA [ceruza elem]()

f) Kb. 10x10x2 cm-es hungarocell lap

g) Hátlap igen, nem és talán válaszokkal pl. [Picúros mintájú]()

## A kapcsolási rajz

<img src="bevezeto_szervo.png" width=600 height=400 />

### Kapcsolási rajz részletező

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Motorok]() című bevezető leírásban található.

1) A szervo motoroknak három kivezetése van. Kettő közülük a motor forgatásáért felel, a harmadik pedig a pozíció vezérléséért felel és nincs összekapcsolva a másik kettővel. Igazából nagy a valószínűsége, hogy bármely szervo esetén, a kimeneti drótok nincsenek direkt összekapcsolva a motorral, hanem egy nekik megfelelő driveren keresztül vannak összekötve. Ettől függetlenül, a szignál drót kilétét úgy tudjuk megállapítani, hogy megnézzük melyik drót nincs összekötve a többivel. Arra a drótra kell a PWM jelet kapcsolni. A másik kettőre elviekben mind a + és - tápot is lehet kötni.

2) Kössük a szignál drótját a Raspberry Pi *GPIO18*-as tüskéjéhez. 

3) Kapcsoljuk a külső tápunkat a szervo maradék két drótjára. 

4) A Raspberry Pi bármelyik földjét kössük össze a táp és a motor negatív (-) drótjával.

5) Illesszünk be egy nyomógombot a breadboardba és annak egyik lábát kapcsoljuk a földelésre a másikat pedig a *GPIO22*-es tüskére.

**Kétszer is ellenőrizzük le, hogy a bekötésünk rendben van-e. A félrekötött tüskék nagyban növelik a motor vagy a Raspberry Pi tökremenési esélyeit. A motort SEMMIKÉPP SE tápláljuk és irányítsük direktben a Raspberry Pi-ről, mert az szinte biztosan a számítógép sérükéséhez vezet.**

### Projekt összeszerelési útmutató

<img src="lucky_wheel_1.jpg" width=600 height=400 />

A szerencsekerék összeszereléséhez a fenti képen látható elemek fognak kelleni. Egy alapzat (hungarocell), egy duncosgumi, egy motor és egy keménylapra ragasztott szerencsekerék papír.

<img src="lucky_wheel_2.jpg" width=600 height=400 />

Kb. 20x20x2 cm-es hungarocell lapot vágjunk ki, aminek nagyjából a közepét kb. 1x1 cm-es méretben átlukasztjuk. A hungarocell egyik sarkába helyezzünk (egyelőre ne ragasszuk oda) egy nyíl alakú jelzést (akár papírból is lehet, a lényeg, hogy csúcsos legyen a lyuk felé). Ez a nyíl fogja jelezni a szerencsekeréken, hogy melyik nyereményt kapjuk. 

<img src="lucky_wheel_3.jpg" width=600 height=400 />

Ezután rögzítsük a motort a hungarocellhez egy duncosgumival. Ha kell használjunk kettő duncosgumit, vagy bármilyen más leszorítást. A lényeg, hogy a motor forgása közben maga a motor teste ne mozogjon, csak a tengelye forogjon. 

<img src="lucky_wheel_4.jpg" width=600 height=400 />

Végül helyezzük a szerencsekerék lapját a motor tengelyére. Győződjünk meg róla, hogy a lap nem lötyög és nem esik le könnyen. Egyben igazítsunk a nyílacska pozícióján ha kell úgy, hogy felülről a lapra nézve egyértelműen lássuk melyik mezőre mutat a nyíl. Ha ez meg van, ragasszuk a nyílat a hungarocellhez.

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```servo_answer.py``` név alatt.

### A szervo motor kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a motor mozgatására.

Először betöltjük a ```gpiozero``` modulból a ```Servo``` objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a motorral a hozzá tartozó vezérlővel a **GPIO** tüskéken keresztül. Még beimportáljuk az ```AngularServo``` objektumot, ami ugyanaz mint a ```Servo```, csak a pozíciót szögekben lehet megadni, valamint a ```Button``` objektumot a gombok érzékeléséhez. 

Továbbá szükségünk lesz:

* az ```unidecode``` csomagra, amit arra használunk, hogy a beírt kérdésekben a magyar ékezetes betűkből eltüntessük az ékezetet,
* a ```sys``` csomagra, hogy a gomb lenyomásával ki tudjunk lépni a pythonból, 
* a ```random``` csomagra, hogy véletlenszerű válaszokat generálhassunk,
* a ```time``` csomagra, hogy szünetet vigyünk a python lefütásába adott esetben.

```servo_answer.py```:

In [ ]:
from gpiozero import Servo, AngularServo, Button
import unidecode  # pip or conda install might be needed
import sys, random
import time

A csomagok és objektumok beolvasása után az első dolgunk a szervo motor letesztelése. Ehhez inicializálunk egy ```Servo``` objektumot és mozgatjuk a minimum és maximum pozíciója között. A szervohoz szükséges jelet a Raspberry *GPIO18*-as tüskéjéhez kötjük, mert az képes hardweres PWM üzemmódban működni. 

```servo_answer.py```:

In [ ]:
from gpiozero import Servo, AngularServo, Button
import unidecode  # pip or conda install might be needed
import sys, random
import time

motor = Servo(18)
motor.min()  # measure angle
motor.max()  # measure angle again

Tegyük a szervo motor mögé a válasz táblát és figyeljük meg, hogy a végpozíciókhoz képest milyen szög értéket kell a mutatónak felvennie, hogy egy adott válaszra mutasson. Ez fontos, hogy az ```AngularServo``` esetén egy adott válasz megjelenítéséhez tudjunk milyen szöget kell társítani. 

Tegyük fel, hogy a motor mozgástartománya a -90 foktól +90 fokig terjed. Rendeljük ezeket a szögeket egy-egy változóhoz és inicializáljunk velük egy ```AngularServo``` objektumot szintén a *GPIO18*-as tüskére. Még mielőtt ezt megtennénk, szüntessük meg a kommunikációt az előbb inicializált szervo motorral a ```motor.close()``` paranccsal, vagy lépjünk ki a pythonból és indítsuk újra.  

```servo_answer.py```:

In [2]:
from gpiozero import Servo, AngularServo, Button
import unidecode  # pip or conda install might be needed
import sys, random
import time

angle_min = -90
angle_max = 90

motor = AngularServo(18, min_angle = angle_min, max_angle=angle_max)
motor.angle = None

A végén rendeljük a motor szögéhez a ```None``` értéket, ez megszünteti a motor pozíció ellenőrzését és az utolsó megadott állapotában tartja a motort. Ha ezt nem tesszük, akkor az SG90-es motor nagyon kis szögben ugrándozik bizonytalanul az utolsó felvett pozíció körül egyfolytában aktívan tartva a motort. Ezért, általánosan érdemes bármely új pozíció felvétele után a ```None``` értéket rendelni a motor szögéhez.

### A gombok tesztelése

Következő lépésben beimportáljuk a gombokért felelős ```Button``` objektumot is és definiálunk egy gombot, amit aa *GPIO22*-re kötjük. 

```servo_answer.py```:

In [ ]:
from gpiozero import Servo, AngularServo, Button
import unidecode  # pip or conda install might be needed
import sys, random
import time

angle_min = -90
angle_max = 90

motor = AngularServo(18, min_angle = angle_min, max_angle=angle_max)
motor.angle = None

button_quit = Button(22)


Ahhoz, hogy leteszteljük helyesen kötöttük-e be a gombot és, hogy működik, írunk egy segédfüggvényt ```test()```, amit a gomb lenyomásakor meghívunk majd. A függvény meghívásakor egyszerűen csak kiíratjuk a képernyőre, hogy a gombot megnyomtuk. Végül ezt a függvényt hozzá kell rendelnünk a gombhoz, méghozzá most a gomb lenyomásának pillanatához a ```when_pressed``` metódushoz.

```servo_answer.py```:

In [ ]:
from gpiozero import Servo, AngularServo, Button
import unidecode  # pip or conda install might be needed
import sys, random
import time

angle_min = -90
angle_max = 90

motor = AngularServo(18, min_angle = angle_min, max_angle=angle_max)
motor.angle = None

button_quit = Button(22)

def test():
    print('Button is pressed')
    
button_quit.when_pressed = test

while True:
    pass

A kód végén még elindítunk egy végtelen ciklust, ami nem csinál semmit, így csak a gombok lenyomása lesz az egyetlen egy esemény amit a számítógép jelezhet. 

Mentsük le a kódot és futtasuk le a pythonnal. Teszteljük, hogy a gombok lenyomására megjelenik-e az üzenet a képernyőn. 

### Válaszokhoz rendelhető szögek definiálása

Elérkeztünk a projekt legfontosabb részéhez, a kerék forgatásához szükséges függvény definiálásához. Nevezzük el, ```start(button, m)``` függvénynek. Kettő bemenő paramétert fogad, az első a gomb, ```button```, amit lenyomunk, a második a motor, ```m```, objektuma.

A függvény egyik feladata, hogy véletlenszerű ideig forgassa a kereket 0 és 5 másodperc között. Ehhez létrehozunk egy új változót, ```t_rot```, ami egy véletlenszerű valós szám értékét veszi fel 0 és 5 között, ```t_rot = random.uniform(0,5)```.

Ugyanezt a módszert használjuk, hogy a függvényünk második feladatát végrehajtsuk, a véletlenszerű sebesség generálását. Ezt is egy változóhoz rendeljük hozzá, ```speed = random.uniform(0.6, 1)```. Itt 0,6-tól indítjuk a generálást, mert kisebb értékeknél lehet, hogy a motornak nehezére esne már forgatni a kereket (habár ez motor függő). 

Ha legeneráltuk a két véletlen számot, már csak alkalmaznunk kell őket. Először elkezdjük forgatni a motort a legenerált sebességgel, ```m.forward(speed=speed)```. Ezután hagyjuk azt forogni a legenerált forgási ideig, ```sleep(t_rot)```. Végül megállítjuk a motort, ```m.stop()```.

In [1]:
def start(button, m):
    t_rot = random.uniform(0, 5)
    speed = random.uniform(0.6, 1)
    m.forward(speed=speed)
    sleep(t_rot)
    m.stop()

A fent definiált függvény általánosan használható tetszőleges gombra és tetszőleges motorra, amiket megadunk bemenő paraméterként. Így ha egy feladatban több motort is kellene evvel a módszerrel forgatni, akkor mindegyikre alkalmazható lenne. 

### A nem megfelelő kérdőszavak listája

Utolsó lépésként az előbb definiált függvényt beillesszük a kódunkba és hozzárendeljük gomb megfelelő metódusához, használva a ```lambda``` kifejezést, hiszen a gombok metódusai mindig függvényeket várnak értékül. Itt a ```.when_pressed``` metódust fogjuk használni, hiszen azt szeretnénk, hogy a gombok lenyomására aktiválódjanak az egyes függvények. A ```lambda``` kifejezés testébe írjuk a ```start``` függvényt, minthogy azt kell végrehajtani, és annak bemenő paraméterként megadjuk a ```button_start``` gombot és a ```motor``` nevű motort. 

```servo_answer.py```:

In [ ]:
from gpiozero import Motor, Button
from time import sleep
import random

motor = Motor(forward=23, backward = 24, enable=25) 
button_start = Button(17)


def start(button, m):
    t_rot = random.uniform(0, 5)
    speed = random.uniform(0.6, 1)
    m.forward(speed=speed)
    sleep(t_rot)
    m.stop()


button_start.when_pressed = lambda : start(button_start, motor)


while True:
    pass

A kód végére beírunk még egy végtelen ciklust, hogy a program lefutása után a python kód ne álljon meg, hanem folyamatosan várja a gomb lenyomásokat. 

### A beírt kérdés tesztelése

### A választ adó függvény definiálása

### A kérdés beolvasása és válasz megjelenítése

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```servo_answer.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```servo_answer.py```-t elmentettük. Ott begépelve a ```python servo_answer.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a start gomb megnyomása után a motor elkezdi forgatni a kereket és egyszercsak, véletlenszerűen megáll. Ha újra megnyomjuk a gombot, akkor más sebességgel más ideig fog forogni.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Írjuk át a kódot úgy, hogy a motor ne hirtelen, hanem fokozatosan lassulva álljon meg.
* Írjuk át a kódot úgy, hogy az egyik gomb elindítja a kerék forgását és egy másik pedig megállítja. Így a játékos maga dönt afölött, hogy mikor állítja meg a kereket.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) Motor objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#motor](https://gpiozero.readthedocs.io/en/stable/api_output.html#motor)

3) L293D driver leírása - [https://www.ti.com/lit/ds/symlink/l293.pdf](https://www.ti.com/lit/ds/symlink/l293.pdf)

4) Button objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_input.html?highlight=button#button](https://gpiozero.readthedocs.io/en/stable/api_input.html?highlight=button#button)

5) lambda kifejezés - [https://www.programiz.com/python-programming/anonymous-function](https://www.programiz.com/python-programming/anonymous-function)

6) random csomag leírása - [https://docs.python.org/3/library/random.html](https://docs.python.org/3/library/random.html)